In [35]:
import joblib
import pandas as pd

# 모델 파일 경로
model_path = "../버전2(SPY)/modeling/Model/다중회귀_best.joblib"  

# 예측 데이터 파일 경로
data_path = "../예측/SPYlog샘플데이터.csv"

try:
    loaded_model = joblib.load(model_path)
    print("✅ 모델 불러오기 완료.")
except FileNotFoundError:
    print(f"❌ 오류: 모델 파일 경로를 찾을 수 없습니다: {model_path}")
    # 프로그램 종료 또는 다음 단계로 이동하지 않음
except Exception as e:
    print(f"❌ 모델 불러오기 중 오류 발생: {e}")

✅ 모델 불러오기 완료.


In [36]:
try:
    df_predict = pd.read_csv(data_path)
    print("✅ 데이터셋 불러오기 완료. 데이터셋 정보:")
    print(df_predict.head())
except FileNotFoundError:
    print(f"❌ 오류: 데이터셋 파일 경로를 찾을 수 없습니다: {data_path}")
except Exception as e:
    print(f"❌ 데이터셋 불러오기 중 오류 발생: {e}")

✅ 데이터셋 불러오기 완료. 데이터셋 정보:
   Unnamed: 0        Date  GC=F_Volume  ^VIX_Volatility  GC=F_Volatility  \
0           0  2025-11-10     5.241747         0.797507         3.947386   
1           1  2025-11-11     6.343880         0.565314         3.505566   
2           2  2025-11-12     5.981414         0.672944         4.620059   
3           3  2025-11-13     5.117994         1.568616         4.174387   
4           4  2025-11-14     6.180017         1.497389         5.016617   

   SHY_Volatility  SPY_Premium_pct  y_target_log  
0        0.029558         0.025857      6.524208  
1        0.039222         0.047590      6.526495  
2        0.029565         0.036686      6.527051  
3        0.029558         0.028242      6.510318  
4        0.086174         0.041819      6.510154  


In [37]:
from sklearn.preprocessing import MinMaxScaler

# -------------------------------
# 1. 전역 변수 설정 및 데이터 준비
# -------------------------------

# [중요] 학습 시 최적으로 도출된 seq_len 값을 직접 입력해야 합니다.
# 예시에서는 임의의 값으로 설정합니다. 실제 값을 입력해주세요!
BEST_SEQ_LEN = 2 

# -------------------------------
# 1. 전역 변수 설정 및 데이터 준비 (수정된 부분)
# -------------------------------

price_col = "y_target_log"

# 'Unnamed: 0', 'y_target_log' 외에 'Date'도 피처에서 제외해야 합니다.
# 예측 데이터에 'Date' 칼럼이 있다면 반드시 drop_cols에 포함해야 합니다.
drop_cols = ["Unnamed: 0", price_col, "Date"] 

# X_predict는 오직 숫자형 피처만 포함해야 합니다.
X_predict = df_predict.drop(columns=drop_cols, errors='ignore') 
y_log_predict = df_predict[price_col]

# [중요] X_predict의 모든 칼럼이 숫자형인지 최종 확인
# (만약 다른 문자열 피처가 있다면 여기서 오류가 다시 발생할 수 있습니다.)
X_predict = X_predict.select_dtypes(include=[np.number])


# X_predict가 학습 시 사용된 MinMaxScaler로 스케일링되어야 합니다.
scaler_seq = MinMaxScaler()

# 학습 데이터와 예측 데이터의 피처 이름이 일치하는지 확인해야 합니다.
# (Date 칼럼 제거 후)
X_predict_scaled = scaler_seq.fit_transform(X_predict.values) 

print(f"✅ 피처 스케일링 완료. Shape: {X_predict_scaled.shape}")

✅ 피처 스케일링 완료. Shape: (7, 5)


In [38]:
# 1-1. 학습 시 사용된 5개 피처 이름을 여기에 입력해야 합니다.
# [사용자 변경 필요]
FEATURE_COLUMNS_FOR_PREDICTION = ['GC=F_Volume', '^VIX_Volatility', 'GC=F_Volatility', 'SHY_Volatility', 'SPY_Premium_pct'] 
# -------------------------------------------------------------

price_col = "y_target_log"
drop_cols = ["Unnamed: 0", price_col, "Date"] 

# 1. 'y_target_log' (로그 레벨) 분리
y_log_predict = df_predict[price_col]

# 2. 피처에서 불필요한 열 제거
X_all = df_predict.drop(columns=drop_cols, errors='ignore')

# 3. [핵심 수정] 학습에 사용된 5개 피처만 선택
#    (FEATURE_COLUMNS_FOR_PREDICTION 리스트를 사용하여 9개 중 5개만 남김)
X_predict = X_all[FEATURE_COLUMNS_FOR_PREDICTION] 

# 4. 숫자형 피처만 최종 확인 (선택된 5개 피처는 모두 숫자형이어야 함)
X_predict = X_predict.select_dtypes(include=[np.number])


# X_predict가 학습 시 사용된 MinMaxScaler로 스케일링되어야 합니다.
scaler_seq = MinMaxScaler()

# 이제 X_predict의 shape는 (7, 5)가 될 것이며, 
# 시퀀스 변환 후 (7-2+1=6) 샘플의 shape는 (6, 5*2=10)이 되어 모델 기대치와 일치합니다.
X_predict_scaled = scaler_seq.fit_transform(X_predict.values) 

print(f"✅ 피처 스케일링 완료. Shape: {X_predict_scaled.shape}")
print(f"✅ 현재 총 피처 수: {X_predict_scaled.shape[1]}") # 5여야 합니다.

✅ 피처 스케일링 완료. Shape: (7, 5)
✅ 현재 총 피처 수: 5


In [39]:
# 필요한 라이브러리/모듈이 이미 상단에 정의되어 있다고 가정합니다.
# 예: import numpy as np, import pandas as pd, loaded_model 변수 존재

# -------------------------------
# 2. 시퀀스 데이터셋 생성 함수 재정의
# -------------------------------
def make_seq_dataset_for_reg(X_array, y_array, seq_len):
    """
    X_array: (N, num_features)
    y_array: (N,)
    seq_len 기간만큼 window를 만들어,
      X_seq: (num_samples, seq_len, num_features)
      y_seq: (num_samples,)
    를 반환합니다. 타깃은 window의 마지막 시점 y.
    """
    X_list = []
    y_list = []
    
    # 예측 데이터의 경우, 모든 시퀀스 생성 가능 시점부터 예측을 수행합니다.
    for i in range(len(X_array) - seq_len + 1):
        X_list.append(X_array[i : i + seq_len])
        # y_array는 예측 시점의 실제 로그 레벨(Ground Truth)입니다.
        y_list.append(y_array[i + seq_len - 1]) 
        
    return np.array(X_list), np.array(y_list)

# -------------------------------
# 3. 시퀀스 변환 및 예측 수행
# -------------------------------

# X_predict_scaled (Shape: (7, 9))와 y_log_predict를 사용합니다.
# BEST_SEQ_LEN = 2
X_seq_predict, y_seq_true = make_seq_dataset_for_reg(
    X_predict_scaled, 
    y_log_predict.values, 
    BEST_SEQ_LEN
)
n_samples = len(X_seq_predict) # 예상 n_samples: 7 - 2 + 1 = 6

if n_samples == 0:
    print(f"❌ 오류: 시퀀스 샘플 수가 0입니다. (데이터 길이:{len(X_predict_scaled)} vs BEST_SEQ_LEN:{BEST_SEQ_LEN})")
else:
    # 3D (n_samples, seq_len, n_features) → 2D (n_samples, seq_len * n_features)
    X_predict_2d = X_seq_predict.reshape(n_samples, -1)
    
    print(f"✅ 시퀀스 데이터셋 재구성 완료.")
    print(f"   예측 샘플 수: {n_samples}")
    print(f"   모델 입력 shape: {X_predict_2d.shape}")

    # 예측 수행 (차분 값 ret_prev 예측)
    # loaded_model은 LinearRegression 모델을 가정합니다.
    y_ret_pred = loaded_model.predict(X_predict_2d)

    print(f"✅ 모델 예측 완료. 예측된 ret_prev 길이: {len(y_ret_pred)}")

    # -------------------------------
    # 4. 차분 → 로그 레벨 복원
    # logP_pred[t] = logP_true[t-1] + ret_prev_pred[t]
    # -------------------------------

    # 4-1. 예측된 시퀀스가 원래 데이터에서 가리키는 인덱스 계산
    # 예측된 첫 시점의 원래 인덱스는 (0 + BEST_SEQ_LEN - 1)
    orig_pos_start = BEST_SEQ_LEN - 1
    orig_pos_end = orig_pos_start + n_samples # 예측된 마지막 시점 인덱스 + 1

    # 4-2. 복원에 필요한 직전 시점의 실제 로그 레벨 값 (log P[t-1])
    # 인덱스 범위: (orig_pos_start - 1) 부터 (orig_pos_end - 1)까지
    log_prev_seq = y_log_predict.iloc[orig_pos_start - 1 : orig_pos_end - 1].to_numpy()

    # 4-3. 예측된 log level 가격 (log P[t]) 복원
    y_log_pred = log_prev_seq + y_ret_pred

    # -------------------------------
    # 5. 최종 결과 DataFrame 생성 및 저장
    # -------------------------------

    # 원래 데이터의 날짜와 로그 레벨 ground truth
    date_predict = df_predict.iloc[orig_pos_start : orig_pos_end]['Date'].reset_index(drop=True)

    result_df = pd.DataFrame({
        "Date": date_predict,
        "log_true": y_seq_true, 
        "log_pred": y_log_pred,
        "Price_pred": np.exp(y_log_pred) # 실제 가격 레벨 복원 (로그의 역변환)
    }).set_index("Date")

    print("\n=== 최종 예측 결과 (로그 및 실제 가격 레벨 - 앞 5개) ===")
    print(result_df.head())

✅ 시퀀스 데이터셋 재구성 완료.
   예측 샘플 수: 6
   모델 입력 shape: (6, 10)
✅ 모델 예측 완료. 예측된 ret_prev 길이: 6

=== 최종 예측 결과 (로그 및 실제 가격 레벨 - 앞 5개) ===
            log_true  log_pred  Price_pred
Date                                      
2025-11-11  6.526495  6.533879  688.062366
2025-11-12  6.527051  6.532389  687.037840
2025-11-13  6.510318  6.502512  666.814325
2025-11-14  6.510154  6.510263  672.003450
2025-11-17  6.500794  6.500677  665.591945


In [41]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import pandas as pd

# 이전에 생성된 DataFrame을 사용합니다. (result_df 변수가 현재 환경에 존재한다고 가정)

# 1. 실제 값(True)과 예측 값(Predicted) 추출
# 로그 레벨 기준으로 성능을 측정합니다.
y_true = result_df['log_true'].to_numpy()
y_pred = result_df['log_pred'].to_numpy()

# 2. 성능 지표 계산

# 2-1. RMSE (Root Mean Squared Error, 제곱 평균 제곱근 오차)
rmse_log = np.sqrt(mean_squared_error(y_true, y_pred))

# 2-2. MAE (Mean Absolute Error, 평균 절대 오차)
mae_log = mean_absolute_error(y_true, y_pred)

# 2-3. R^2 Score (Coefficient of Determination, 결정 계수)
r2_log = r2_score(y_true, y_pred)

# 3. 결과 출력
print("\n=== 예측 모델 성능 지표 (로그 레벨 기준) ===")
print(f"**RMSE (Root Mean Squared Error)**: {rmse_log:.6f}")
print(f"**MAE (Mean Absolute Error)** : {mae_log:.6f}")
print(f"**R^2 Score (결정 계수)** : {r2_log:.6f}")
print("=================================================")


=== 예측 모델 성능 지표 (로그 레벨 기준) ===
**RMSE (Root Mean Squared Error)**: 0.005309
**MAE (Mean Absolute Error)** : 0.004295
**R^2 Score (결정 계수)** : 0.822022
